In [1]:
#!pip install -r requirements.txt

In [2]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
import os
settings.update({'datasets_dir': os.path.abspath('.')})


Install mish-cuda to speed up training and inference. More importantly, replace the naive Mish with MishCuda will give a ~1.5G memory saving during training.
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name bei

In [3]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['NWPU_VHR10_YOLO', 'COCO_airplane',]
model_names = ['yolov8n','YoloOW_CLI']
iterations = [1,2]
epochs = 30
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 0.001 
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 0

In [4]:
from utility.utils import control_random_seed, update_dataset_paths, format_measures, merge_and_update_df


Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers
ex_dict['AutoAnchor'] = True

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        # 클래스별 메트릭 초기화 - 이전 데이터셋의 클래스별 메트릭이 남아있는 문제 해결
        for key in list(ex_dict.keys()):
            if '/' in key:  # 클래스별 메트릭은 'class_name/metric_name' 형식
                del ex_dict[key]
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn, test_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)
            ex_dict = test_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            #################
            if ex_dict.get('Test Results') is None:
                raise RuntimeError("평가 결과가 없습니다. 위의 체크리스트를 참고해 원인부터 해결하세요.")

            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

Experiment Start Time: 250527_170914
(Iter 1)
Dataset: NWPU_VHR10_YOLO (1/2)
yolov8n (1/3) (Iter 1) Dataset: NWPU_VHR10_YOLO (1/2) WARNING  no model scale passed. Assuming scale='n'.
New https://pypi.org/project/ultralytics/8.3.145 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=Models/ultralytics/ultralytics/cfg/models/v8/yolov8.yaml, data=Datasets/NWPU_VHR10_YOLO/data_iter_01.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=output, name=250527_170914_yolov8n_NWPU_VHR10_YOLO_iteration1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=

train: Scanning Datasets\NWPU_VHR10_YOLO\labels... 390 images, 0 backgrounds, 0 corrupt: 100%|██████████| 390/390 [00:00<00:00, 1360.71it/s]

train: New cache created: Datasets\NWPU_VHR10_YOLO\labels.cache



val: Scanning Datasets\NWPU_VHR10_YOLO\labels... 130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 1206.94it/s]

val: New cache created: Datasets\NWPU_VHR10_YOLO\labels.cache


...

[YoloOW] Epoch 14/30:  96%|█████████▌| 23/24 [00:23<00:01,  1.34s/it, Loss=5.8336, box=0.077, obj=0.037, cls=0.250, total loss=0.365, lr=0.000647]

[DEBUG] Head 0 | iou.min=0.1171, iou.max=0.8836, iou.mean=0.4647
[DEBUG] Head 0 | ps[..., 4].sigmoid().mean = 0.0071
[DEBUG] Head 1 | iou.min=0.0590, iou.max=0.8585, iou.mean=0.3846
[DEBUG] Head 1 | ps[..., 4].sigmoid().mean = 0.0112
[DEBUG] Head 2 | iou.min=-0.1237, iou.max=0.9133, iou.mean=0.3222
[DEBUG] Head 2 | ps[..., 4].sigmoid().mean = 0.0177


[YoloOW] Epoch 14/30: 100%|██████████| 24/24 [00:25<00:00,  1.06s/it, Loss=5.5830, box=0.091, obj=0.047, cls=0.211, total loss=0.349, lr=0.000647]



[YoloOW] ✅ Epoch 14/30 complete in 25.51s | Avg Loss: 5.4991

[YoloOW] 📊 Running evaluation after epoch 14 ...


KeyboardInterrupt: 

In [5]:
import torch

print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
print(f"현재 사용 디바이스: {torch.cuda.current_device()}")
print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
print(f"PyTorch CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.__version__}")


CUDA 사용 가능 여부: True
CUDA 디바이스 수: 1
현재 사용 디바이스: 0
디바이스 이름: NVIDIA GeForce RTX 5060 Ti
PyTorch CUDA 버전: 12.8
PyTorch 버전: 2.8.0.dev20250420+cu128
